In [ ]:
import logging
import logging.config
import warnings

import iris
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from joblib import Memory, Parallel, delayed
from matplotlib import animation, rc
from tqdm import tqdm, tqdm_notebook

from wildfires.analysis.plotting import cube_plotting, get_cubes_vmin_vmax
from wildfires.data.cube_aggregation import Datasets, get_ncpus, prepare_selection
from wildfires.data.datasets import DATA_DIR, GSMaP_dry_day_period, data_map_plot
from wildfires.logging_config import LOGGING
from wildfires.utils import Time, TqdmContext
from wildfires.utils import land_mask as get_land_mask
from wildfires.utils import match_shape, polygon_mask

logger = logging.getLogger(__name__)
logging.config.dictConfig(LOGGING)

# tqdm_notebook does not work for some reason

warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")
memory = Memory(DATA_DIR)

In [ ]:
dry_day_period = Datasets(GSMaP_dry_day_period())
print(dry_day_period.dataset.cube)

In [ ]:
min_value = dry_day_period.dataset.cube.collapsed(
    ["time", "latitude", "longitude"], iris.analysis.MIN
)
max_value = dry_day_period.dataset.cube.collapsed(
    ["time", "latitude", "longitude"], iris.analysis.MAX
)
mean_periods = dry_day_period.dataset.cube.collapsed("time", iris.analysis.MEAN)
min_periods = dry_day_period.dataset.cube.collapsed("time", iris.analysis.MIN)
max_periods = dry_day_period.dataset.cube.collapsed("time", iris.analysis.MAX)

# Improve performance when accessing lazy data by grouping calculations like this.
iris.cube.CubeList(
    [min_value, max_value, mean_periods, min_periods, max_periods]
).realise_data()

print("Min dry days:\n{}\n{}".format(min_value, min_value.data))
print("Max dry days:\n{}\n{}".format(max_value, max_value.data))

mpl.rcParams["figure.figsize"] = (10, 4)
for data, title in zip(
    (mean_periods, min_periods, max_periods),
    ("Mean Dry Day Period", "Min Dry Day Period", "Max Dry Day Period"),
):
    cube_plotting(data, log=True, title=title, auto_log_title=True)

In [ ]:
(
    monthly_dry_day_period,
    mean_dry_day_period,
    climatology_dry_day_period,
) = prepare_selection(dry_day_period.copy(deep=True))

# Define and use relevant masks.
land_mask = ~get_land_mask()

# Define a latitude mask which ignores data beyond 60 degrees, as the precipitation data does not extend to those latitudes.
lat_mask = ~polygon_mask([(180, -60), (-180, -60), (-180, 60), (180, 60), (180, -60)])

# Apply the masks.
monthly_dry_day_period.dataset.cube.data.mask |= match_shape(
    land_mask, monthly_dry_day_period.dataset.cube.shape
) | match_shape(lat_mask, monthly_dry_day_period.dataset.cube.shape)

In [ ]:
warnings.filterwarnings("ignore", ".*divide by zero.*")
warnings.filterwarnings("ignore", ".*invalid value encountered.*")

mpl.rcParams["figure.figsize"] = (14, 9)

# Define a latitude mask which ignores data beyond 60 degrees, as the precipitation data does not extend to those latitudes.
lat_bounds = monthly_dry_day_period.dataset.cube.coord("latitude").contiguous_bounds()
lon_bounds = monthly_dry_day_period.dataset.cube.coord("longitude").contiguous_bounds()
n_lat = len(lat_bounds) - 1
n_lon = len(lon_bounds) - 1

vmin, vmax = get_cubes_vmin_vmax(monthly_dry_day_period.cubes)


@memory.cache
def get_js_animation(N_frames=monthly_dry_day_period.dataset.cube.shape[0]):
    fig, ax, mesh, suptitle_text = cube_plotting(
        monthly_dry_day_period.dataset.cube[0],
        log=True,
        animation_output=True,
        title="",
        vmin=vmin,
        vmax=vmax,
    )
    title_text = ax.text(
        0.5, 1.08, "bla", transform=ax.transAxes, ha="center", fontsize=15
    )
    plt.close()  # Prevent display of (duplicate) static figure due to %matplotlib inline

    # N_frames = len(new_dataset)
    # N_frames = 4
    interval = 1000 / 12  # One second per year.

    def init():
        mesh.set_array(np.zeros(n_lat * n_lon))
        title_text.set_text("")
        return (mesh,)

    with TqdmContext(unit=" plots", desc="Plotting", total=N_frames) as t:

        def animate(i):
            single_time_cube = monthly_dry_day_period.dataset.cube[i]
            _ = cube_plotting(
                single_time_cube,
                log=True,
                ax=ax,
                mesh=mesh,
                animation_output=False,
                new_colorbar=False,
                title="",
            )
            title_text.set_text(
                # Ignore the time, which flip-flops between 00:00:00 and 12:00:00.
                "Dry Day Period "
                + str(single_time_cube.coord("time").cell(0).point)[:10]
            )
            t.update()

            return (mesh, title_text)

        anim = animation.FuncAnimation(
            fig, animate, init_func=init, frames=N_frames, interval=interval, blit=True
        )

        js_output = anim.to_jshtml()
        return js_output


HTML(get_js_animation())